In [ ]:
import autograd.numpy as np
from autograd import grad, elementwise_grad
import autograd.numpy.random as npr
from autograd.misc.optimizers import adam

In [ ]:
x0 = 0
x1 = 1

def init_random_params(scale, layer_sizes, rs=npr.RandomState(42)):
    return [(rs.randn(insize, outsize) * scale,   # weight matrix
             rs.randn(outsize) * scale)           # bias vector
            for insize, outsize in zip(layer_sizes[:-1], layer_sizes[1:])]

def tanh(x):
    return (2 / (1.0 + np.exp(-2*x))) - 1

def psi(nnparams, inputs):
    origInput = inputs
    xTilde = (origInput - x0) / (x1 - x0)
    
    for W, b in nnparams:
        outputs = np.dot(inputs, W) + b
        inputs = tanh(outputs)    
        
    return (1-np.exp(xTilde * (1 - xTilde))) * outputs

dpsi = elementwise_grad(psi, 1) # dpsi/dx 
ddpsi = elementwise_grad(dpsi, 1) # d^2psi/dx^2

In [ ]:
nnparams = init_random_params(2, layer_sizes=[1, 32, 32, 1])
params = {'nn': nnparams, 'E': 19.1}

x = np.linspace(0, 1, 200)[:, None]

def objective(params, step):
    nnparams = params['nn']
    E = params['E']        
    
    wave = psi(nnparams,x)
    zeq = -0.5 * ddpsi(nnparams, x)  - E*wave
    y2 = wave**2
    
    # This is a numerical trapezoid integration
    prob = np.sum((y2[1:] + y2[0:-1]) / 2 * (x[1:] - x[0:-1]))
    
    return np.mean(zeq**2)

def objectiveDebug (params, step) :
    nnparams = params['nn']
    E = params['E']        
    # This is Schrodinger's eqn
    zeq = -0.5 * ddpsi(nnparams, x)  - E*psi(nnparams, x) 
    y2 = psi(nnparams, x)**2
    # This is a numerical trapezoid integration
    prob = np.sum((y2[1:] + y2[0:-1]) / 2 * (x[1:] - x[0:-1]))
    mean = np.mean(zeq**2)
    
    print ("Iteration " + str(step) + " diffeq = " + str(mean) + " prob = " + str(prob))
    print ("loss = " + str(mean + (1-prob)**2) + " E = " + str(params['E']))
    print ("---------------------------------")
    

# This gives us feedback from the optimizer
def callback(params, step, g):
    if step % 100 == 0:
        #print("Iteration {0:3d} objective {1}".format(step,
                                                      #objective(params, step))
        objectiveDebug (params, step)

In [ ]:
params = adam(grad(objective), params,
              step_size=0.001, num_iters=10001, callback=callback) 

print(params['E'])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

x = np.linspace(0, 1)[:, None]
y = psi(params['nn'], x)

plt.plot(x, y, label='NN')
plt.plot(x, np.sqrt(2) * np.sin(2*np.pi * x), 'r--', label='analytical')
plt.legend()

In [ ]:
print (min(y))